In [2]:
addpath ("../files")
setenv('GNUTERM','qt')
format long

In [38]:
%%file ../files/metodoblinterpol2.m

function [W,x,iter] = metodoblinterpol2(fname, x, method='newton', line_search='quadratic',
                                        plot_iterations = true, tol=1.e-04, maxiter = 50, 
                                        maxjter = 30, c1 = 0.1)
%{
    Método de búsqueda de línea con la primer condición de Wolfe
    usando máximo descenso.
    usando los métodos de descenso:
        1. Máximo descenso
        2. Método de Newton
    Input:
        
        fname.- cadena de caracteres con el nombre de la función a minimizar.
        x    .- vector n-dimensional.
    
    Output:
        x    .- vector n-dimensional con la aproximación al mínimo local.
        iter .- contador con el número final de iteraciones externas.
%}
    
    iter = 0;        
    p = 1; s = -1;

    fx = feval(fname,x);
    g  = gradiente(fname,x);
    ng = norm(g);
    W  = [x]; 

    while ( ng > tol && iter < maxiter &&  norm(p)> 1.e-04)
        if (strncmp(method,'max',3))
            p = -g;
        end

        if (strncmp(method,'newton',6))
            H = hessian(fname,x);
            p = -H\g; 
        end

        %-----------------------------------------
        %            Búsqueda de línea
        
        alfa  = 1;              
        xt = x + alfa*p;        
        f  = feval(fname,x);  
        f1 = feval(fname, xt);
        s  = p'*g;            
        jter = 0;             

        
        if (abs(s) < 1.e-06)
            %disp('No existe suficiente descenso  ')
            %disp(sprintf('%2.0f  %2.10f',iter, s   ))
            iter = maxiter; 
        end
        

        while( (f1>f+alfa*c1*s) && jter < maxjter)  % búsqueda de línea
            if (strncmp(line_search,'linear',6))
                alfa = alfa/2;
                xt = x + alfa*p;
                f1 = feval(fname, xt);
                jter = jter +1;
            end
            
            if (strncmp(line_search,'quadratic',9))
                d2 = f1 - f - s;
                alfa = -( s)/(2*d2);       
                f1 = feval(fname, x + alfa*p);
                jter = jter +1;    
            end
        end   
        
        %          Fin de búsqueda de línea
        %------------------------------------------
        
        
        %------------------------------------------
        %                Graficación

        if (plot_iterations == true)
        
            t = linspace(0,1,50)';
            ft = zeros(50,1); rt = zeros(50,1);
            for k = 1:50        
                ft(k) = feval(fname, x+t(k)*p);  % función 
                rt(k) = fx + t(k)*(c1*p'*g);
            end

            fx = feval(fname,x+alfa*p);

            plot(t,ft,'--b',t,rt,'--m',alfa,fx,'dr', 'LineWidth',3)
            title('Gráfica de búsqueda de línea','Fontsize',16)
            xlabel('EJE  T','Fontsize',16)
            ylabel(' f(x + tp)','Fontsize',16  )
            legend('f(x)','recta','punto')
            pause
        end

        %              Fin de graficación
        %-----------------------------------------
          
        W = [W x];
        x = x + alfa*p;  
        fx = feval(fname,x); 
        g = gradiente(fname,x);
        iter = iter + 1;
        
    end      
end


Created file '/Users/sergio_arnaud/Documents/ITAM/8vo/applied_analysis/Labs/files/metodoblinterpol2.m'.


In [ ]:
%plot gnuplot
[W,x,iter] = metodoblinterpol2('rosenbrock',[2.5,2.5]','newton','quadratic',false);

In [ ]:
%plot gnuplot
metodoblinterpol2('fcuad',[2.5,2.5]','newton','linear',true);

## Testing with Shubert function

#### Shubert function

$ f : \mathbb{R}^n \to \mathbb{R}$

$f(\mathbf{x})=\prod_{i=1}^{n}{\left(\sum_{j=1}^5{ cos((j+1)x_i+j)}\right)}$

In [5]:
function [ans] = shubert(X)
    if (length(size(X)) > 2)
        x = X(:,:,1);
        y = X(:,:,2);
    else
        x = X(1);
        y = X(2);
    end

    n = length(x);
    sum1 = zeros(1,n);
    sum2 = zeros(1,n);
    
    for i = 1:5
        sum1 = sum1 + i .* cos((i+1).*x+i);
        sum2 = sum2 + i .* cos((i+1).*y+i);
    end

    ans = sum1 .* sum2;
end

#### Surface plot of Shubert function

In [23]:
x = linspace(-5,5,120);
y = linspace(-5,5,120);
[X,Y] = meshgrid(x,y);
surf(X,Y,shubert(cat(3,X,Y)),'FaceAlpha',0.4)
title('Sergio Arnaud Gómez','Fontsize',16)
legend('Schubert function','recta','punto')


    legend at line 468 column 13


#### Searching for minima

In [48]:
descent_methods = ["newton";"max"];
interpolations = ["linear";"quadratic"];
initial_values = cat(3,[0,0],[1,2],[3,1],[1,3]);

for i = 1:2
    for j = 1:2
        for k = 1:4
            method = descent_methods(i,:);
            value = initial_values(:,:,k);
            interpolation = interpolations(j,:);
            fprintf('Method: %s \nInterpolation: %s \nInitial value: (%d,%d)\n',method, interpolation, value)
            
            [W,x,iter] = metodoblinterpol2('shubert',value',method,interpolation,false);
            fprintf('Value of function %0.5f \n\n',shubert(x))
            
        end
    end
end

Method: newton 
Interpolation: linear    
Initial value: (0,0)
Value of function 0.00000 

Method: newton 
Interpolation: linear    
Initial value: (1,2)
Value of function 0.00000 

Method: newton 
Interpolation: linear    
Initial value: (3,1)
Value of function 0.00000 

Method: newton 
Interpolation: linear    
Initial value: (1,3)
Value of function 0.00000 

Method: newton 
Interpolation: quadratic 
Initial value: (0,0)
Value of function 0.00000 

Method: newton 
Interpolation: quadratic 
Initial value: (1,2)
Value of function 0.00000 

Method: newton 
Interpolation: quadratic 
Initial value: (3,1)
Value of function 0.00000 

Method: newton 
Interpolation: quadratic 
Initial value: (1,3)
Value of function 0.00000 

Method: max    
Interpolation: linear    
Initial value: (0,0)
Value of function 0.00000 

Method: max    
Interpolation: linear    
Initial value: (1,2)
Value of function -46.51131 

Method: max    
Interpolation: linear    
Initial value: (3,1)
Value of function -186.73